[with pytorch](https://gist.github.com/PetrochukM/afaa3613a99a8e7213d2efdd02ae4762)

[notebook](https://github.com/napsternxg/pytorch-practice/blob/master/Viterbi%20decoding%20and%20CRF.ipynb)

[with different rings](https://www.audiolabs-erlangen.de/resources/MIR/FMP/C5/C5S3_Viterbi.html)

[python only?](https://stackoverflow.com/questions/9729968/python-implementation-of-viterbi-algorithm)

[numpy](http://www.adeveloperdiary.com/data-science/machine-learning/implement-viterbi-algorithm-in-hidden-markov-model-using-python-and-r/)

In [9]:

import numpy as np
np.random.seed(2017)

import torch
torch.manual_seed(2017)

# from scipy.misc import logsumexp # Use it for reference checking implementation

In [10]:
seq_length, num_states=4, 2
emissions = np.random.randint(20, size=(seq_length,num_states))*1.
transitions = np.random.randint(10, size=(num_states, num_states))*1.
print("Emissions:", emissions, sep="\n")
print("Transitions:", transitions, sep="\n")

Emissions:
[[ 9.  6.]
 [13. 10.]
 [ 8. 18.]
 [ 3. 15.]]
Transitions:
[[7. 8.]
 [0. 8.]]


In [11]:
def viterbi_decoding(emissions, transitions):
    # Use help from: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/crf/python/ops/crf.py
    scores = np.zeros_like(emissions)
    back_pointers = np.zeros_like(emissions, dtype="int")
    scores = emissions[0]
    # Generate most likely scores and paths for each step in sequence
    for i in range(1, emissions.shape[0]):
        score_with_transition = np.expand_dims(scores, 1) + transitions
        scores = emissions[i] + score_with_transition.max(axis=0)
        back_pointers[i] = np.argmax(score_with_transition, 0)
    # Generate the most likely path
    viterbi = [np.argmax(scores)]
    for bp in reversed(back_pointers[1:]):
        viterbi.append(bp[viterbi[-1]])
    viterbi.reverse()
    viterbi_score = np.max(scores)
    return viterbi_score, viterbi

In [12]:
viterbi_decoding(emissions, transitions)

(78.0, [0, 0, 1, 1])

In [ ]:
emissions = emissions.tolist()
transitions = transitions.tolist()

In [16]:
emissions.shape

(4, 2)

In [ ]:
def viterbi_decoding(emissions, transitions):
    # Use help from: https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/crf/python/ops/crf.py
    list(list(0 for _ in range(len(h)+1)) for _ in range(len(h)+1))
    scores = np.zeros_like(emissions)
    back_pointers = np.zeros_like(emissions, dtype="int")
    scores = emissions[0]
    # Generate most likely scores and paths for each step in sequence
    for i in range(1, emissions.shape[0]):
        score_with_transition = np.expand_dims(scores, 1) + transitions
        scores = emissions[i] + score_with_transition.max(axis=0)
        back_pointers[i] = np.argmax(score_with_transition, 0)
    # Generate the most likely path
    viterbi = [np.argmax(scores)]
    for bp in reversed(back_pointers[1:]):
        viterbi.append(bp[viterbi[-1]])
    viterbi.reverse()
    viterbi_score = np.max(scores)
    return viterbi_score, viterbi

In [ ]:
# https://martin-thoma.com/word-error-rate-calculation/


def levenshtein_distance(r: str, h: str):

    # initialisation
    dold = list(range(len(h)+1))
    dnew = list(0 for _ in range(len(h)+1))

    # computation
    for i in range(1, len(r)+1):
        dnew[0] = i
        for j in range(1, len(h)+1):
            if r[i-1] == h[j-1]:
                dnew[j] = dold[j-1]
            else:
                substitution = dold[j-1] + 1
                insertion = dnew[j-1] + 1
                deletion = dold[j] + 1
                dnew[j] = min(substitution, insertion, deletion)

        dnew, dold = dold, dnew

    return dold[-1]